# Data Science Test


## 1. Evaluate the completeness of data


In [250]:
# Import the necessary libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from ydata_profiling import ProfileReport
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


In [267]:
# Load the data from farms.csv
df_farm_raw = pd.read_csv('data/farms.csv')

# Data profiling for farm.csv
farm_profile = ProfileReport(df_farm_raw, title="Farm Data Profiling Report")
# farm_profile.to_file("profile_data/farm_profile.html")


In [268]:
# Data correction for farm.csv
df_farms = df_farm_raw.dropna().drop_duplicates(subset='id')
cleaned_farm_profile = ProfileReport(df_farm, title="Farm Data Profiling Report")
# cleaned_farm_profile.to_file("profile_data/cleaned_farm_profile.html")


In [265]:
# Load the data from ponds.csv
df_ponds_raw = pd.read_csv('data/ponds.csv')

# Data profiling for ponds.csv
ponds_profile = ProfileReport(df_ponds_raw, title="Ponds Data Profiling Report")
# ponds_profile.to_file("profile_data/ponds_profile.html")


In [266]:
# Clean the data from ponds.csv
df_ponds = df_ponds_raw.dropna(subset=['id', 'length', 'width', 'deep', 'farm_id']).drop_duplicates(subset='id')
df_ponds = df_ponds[['id', 'length', 'width', 'deep', 'farm_id']]
cleaned_ponds_profile = ProfileReport(df_ponds, title="Ponds Data Profiling Report")
# cleaned_ponds_profile.to_file("profile_data/cleaned_ponds_profile.html")


In [236]:
# Load the data from cycles.csv
df_cycles_raw = pd.read_csv('data/cycles.csv')

# Data profiling for cycles.csv
cycles_profile = ProfileReport(df_cycles_raw, title="Cycles Data Profiling Report")
# cycles_profile.to_file("profile_data/cycles_profile.html")


In [269]:
# Clean the data from cycles.csv
df_cycles = df_cycles_raw[['id', 'pond_id', 'started_at', 'finished_at', 'total_seed', 'area']].dropna()
cleaned_cycles_profile = ProfileReport(df_cycles, title="Cycles Data Profiling Report")
# cleaned_cycles_profile.to_file("profile_data/cleaned_cycles_profile.html")


In [270]:
# Load the data from feeds.csv
df_feed_raw = pd.read_csv('data/feeds.csv')

# Data profiling for feeds.csv
feed_profile = ProfileReport(df_feed_raw, title="Feeds Data Profiling Report")
# feed_profile.to_file("profile_data/feeds_profile.html")


In [271]:
# Clean the data from feeds.csv
df_feeds = df_feed_raw.dropna()
cleaned_feeds_profile = ProfileReport(df_feeds, title="Feeds Data Profiling Report")
# cleaned_feeds_profile.to_file("profile_data/cleaned_feeds_profile.html")


In [201]:
# Load the data from fastings.csv
df_fastings_raw = pd.read_csv('data/fastings.csv')

# Data profiling for fastings.csv
fastings_profile = ProfileReport(df_fastings_raw, title="Fastings Data Profiling Report")
# fastings_profile.to_file("profile_data/fastings_profile.html")


In [202]:
# Clean the data from fastings.csv
df_fastings = df_fastings_raw.dropna()
cleaned_fastings_profile = ProfileReport(df_fastings, title="Fastings Data Profiling Report")
# cleaned_fastings_profile.to_file("profile_data/cleaned_fastings_profile.html")


In [238]:
# Load the data from harvests.csv
df_harvest_raw = pd.read_csv('data/harvests.csv')

# Data profiling for harvests.csv
harvest_profile = ProfileReport(df_harvest_raw, title="Harvests Data Profiling Report")
# harvest_profile.to_file("profile_data/harvests_profile.html")


In [204]:
# Clean the data from harvests.csv
df_harvests = df_harvest_raw[['cycle_id', 'harvested_at', 'status', 'size', 'weight', 'selling_price']].dropna()
cleaned_harvests_profile = ProfileReport(df_harvests, title="Harvests Data Profiling Report")
# cleaned_harvests_profile.to_file("profile_data/cleaned_harvests_profile.html")


In [205]:
# Load the data from measurements.csv
df_measurement_raw = pd.read_csv('data/measurements.csv')

# Data profiling for measurements.csv
measurement_profile = ProfileReport(df_measurement_raw, title="Measurements Data Profiling Report")
# # measurement_profile.to_file("profile_data/measurements_profile.html")


In [206]:
# Clean the data from measurements.csv
df_measurement = df_measurement_raw[['cycle_id', 'measured_date', 'morning_temperature', 'evening_temperature', 'morning_do', 'evening_do', 'morning_salinity', 'evening_salinity', 'morning_pH', 'evening_pH', 'transparency', 'ammonia', 'nitrate', 'nitrite', 'alkalinity', 'hardness', 'calcium', 'magnesium', 'carbonate', 'bicarbonate', 'tom', 'total_plankton_']]
cleaned_measurement_profile = ProfileReport(df_measurement, title="Measurements Data Profiling Report")
# cleaned_measurement_profile.to_file("profile_data/cleaned_measurements_profile.html")


In [207]:
# Load the data from samplings.csv
df_sampling_raw = pd.read_csv('data/samplings.csv')

# Data profiling for samplings.csv
sampling_profile = ProfileReport(df_sampling_raw, title="Samplings Data Profiling Report")
# sampling_profile.to_file("profile_data/samplings_profile.html")


In [208]:
# Clean the data from samplings.csv
df_sampling = df_sampling_raw[['cycle_id', 'sampled_at', 'average_weight']]
cleaned_sampling_profile = ProfileReport(df_sampling, title="Samplings Data Profiling Report")
# cleaned_sampling_profile.to_file("profile_data/cleaned_samplings_profile.html")


## 2. Calculate the Survival Rate (SR) and Average Daily Gain (ADG) for each cycle


In [298]:
# Merge the data from cycles.csv and harvests.csv
df_merged = pd.merge(df_cycles, df_harvests, left_on='id', right_on='cycle_id', how='right')

In [305]:
# Group data by 'cycle_id'
grouped_df = df_merged.groupby(['cycle_id', 'pond_id']).agg({
    'size': 'sum',
    'total_seed': 'sum',
    'weight': 'sum',
    'started_at': 'min',
    'harvested_at': 'max'
}).reset_index()

# Convert the 'started_at' and 'harvested_at' columns to datetime format
grouped_df['started_at'] = pd.to_datetime(grouped_df['started_at'])
grouped_df['harvested_at'] = pd.to_datetime(grouped_df['harvested_at'])

# Calculate the difference in days between 'harvested_at' and 'started_at'
grouped_df['days'] = (grouped_df['harvested_at'] - grouped_df['started_at']).dt.days + 1

# Calculate the survival rate (SR) and average daily gain (ADG)
grouped_df['SR'] = (grouped_df['size'] * grouped_df['weight'] / grouped_df['total_seed']) * 100
grouped_df['ADG'] = grouped_df['weight'] / grouped_df['days']

# Select the relevant columns
df_sr_adg = grouped_df[['cycle_id', 'pond_id', 'size', 'total_seed', 'SR', 'weight', 'started_at', 'harvested_at', 'days', 'ADG']]

cycle_id               float64
pond_id                  int64
size                   float64
total_seed               int64
weight                 float64
started_at      datetime64[ns]
harvested_at    datetime64[ns]
days                     int64
SR                     float64
ADG                    float64
dtype: object

In [261]:
# Filter out rows where SR is greater than 100%
df_sr_adg_cleaned = df_sr_adg[df_sr_adg['SR'] <= 100]

# Display the cleaned DataFrame
df_sr_adg_cleaned

,cycle_id,pond_id,size,total_seed,SR,weight,started_at,harvested_at,days,ADG
2,4313.0,12228,333.0,310536,2.841700,26.50,2020-05-05,2020-06-07,34,0.779412
3,4938.0,9603,160.0,75000,71.253333,334.00,2020-07-05,2020-08-28,55,6.072727
4,5222.0,16224,261.0,1256880,77.343324,3724.57,2020-08-01,2020-12-14,136,27.386544
5,5223.0,13378,394.0,2604000,90.054419,5951.82,2020-08-01,2020-12-14,136,43.763382
6,5435.0,16048,246.0,703500,43.421186,1241.74,2020-08-11,2020-10-07,58,21.409310
...,...,...,...,...,...,...,...,...,...,...
1913,29579.0,35864,387.0,100000,94.234500,243.50,2024-02-15,2024-03-26,41,5.939024
1914,29597.0,45690,150.0,400000,46.875000,1250.00,2024-02-18,2024-03-29,41,30.487805
1916,29659.0,45493,200.0,75000,16.000000,60.00,2024-02-11,2024-03-14,33,1.818182
1917,29873.0,43651,85.0,125000,70.584000,1038.00,2024-01-09,2024-03-25,77,13.480519


In [324]:
# Merge the data from ponds.csv and df_sr_adg_cleaned
df_merged_ponds = pd.merge(df_ponds, df_sr_adg_cleaned, left_on='id', right_on='pond_id', how='right')

# Merge the data from farms.csv and df_merged_ponds
df_merged_farms = pd.merge(df_farms, df_merged_ponds, left_on='id', right_on='farm_id', how='right')
df_merged_farms = df_merged_farms[['farm_id', 'pond_id', 'cycle_id', 'province', 'regency', 'size', 'total_seed', 'SR', 'weight', 'started_at', 'harvested_at', 'days', 'ADG']]

# Select the relevant columns
df_sr = df_merged_farms[['pond_id', 'province', 'regency', 'size', 'total_seed', 'SR']]
df_adg = df_merged_farms[['pond_id','province', 'regency', 'weight', 'days', 'ADG']]

In [308]:
# Export to CSV
df_sr.to_csv('output/survival_rate.csv', index=False)
df_adg.to_csv('output/average_daily_gain.csv', index=False)

In [309]:
# View the DataFrame of Survival Rate (SR)
df_sr_with_farm = df_sr.dropna()
df_adg_with_farm = df_adg.dropna()

In [310]:
# View the DataFrame of Survival Rate (SR)
print("Survival Rate (SR)")
df_sr_with_farm

Survival Rate (SR)


,pond_id,province,regency,size,total_seed,SR
1264,44566,JAWA TIMUR,PACITAN,353.0,320000,69.971219
1265,44516,JAWA TIMUR,PACITAN,375.0,210000,71.169643
1266,44517,JAWA TIMUR,PACITAN,309.0,140000,99.045536
1267,44585,JAWA TIMUR,PACITAN,324.0,160000,88.816500
1268,44588,JAWA TIMUR,PACITAN,170.0,120000,39.950000
1269,44586,JAWA TIMUR,PACITAN,316.0,225000,72.174400
1270,44587,JAWA TIMUR,PACITAN,370.0,225000,75.759556
1390,44544,SULAWESI BARAT,MAMUJU UTARA,330.0,362738,59.133590
1392,44548,SULAWESI BARAT,MAMUJU UTARA,468.0,1148588,98.946404


In [311]:
# View the DataFrame of Average Daily Gain (ADG)
print("Average Daily Gain (ADG)")
df_adg_with_farm

Average Daily Gain (ADG)


,pond_id,province,regency,weight,days,ADG
1264,44566,JAWA TIMUR,PACITAN,634.30,72,8.809722
1265,44516,JAWA TIMUR,PACITAN,398.55,72,5.535417
1266,44517,JAWA TIMUR,PACITAN,448.75,72,6.232639
1267,44585,JAWA TIMUR,PACITAN,438.60,71,6.177465
1268,44588,JAWA TIMUR,PACITAN,282.00,65,4.338462
1269,44586,JAWA TIMUR,PACITAN,513.90,71,7.238028
1270,44587,JAWA TIMUR,PACITAN,460.70,71,6.488732
1390,44544,SULAWESI BARAT,MAMUJU UTARA,650.00,28,23.214286
1392,44548,SULAWESI BARAT,MAMUJU UTARA,2428.39,61,39.809672


## 3. Create Predictive Model

In [325]:
# Load the data from harvests.csv and merge with df_merged_farms
df_sell_price = df_harvests[['cycle_id', 'status', 'selling_price']]
df = pd.merge(df_merged_farms, df_sell_price, left_on='cycle_id', right_on='cycle_id', how='right')
df.dropna()

,farm_id,pond_id,cycle_id,province,regency,size,total_seed,SR,weight,started_at,harvested_at,days,ADG,status,selling_price
74,12530.0,44516.0,27148.0,JAWA TIMUR,PACITAN,375.0,210000.0,71.169643,398.55,2023-11-01,2024-01-11,72.0,5.535417,Full,930000.0
75,12530.0,44516.0,27148.0,JAWA TIMUR,PACITAN,375.0,210000.0,71.169643,398.55,2023-11-01,2024-01-11,72.0,5.535417,Full,14454000.0
145,12545.0,44588.0,27151.0,JAWA TIMUR,PACITAN,170.0,120000.0,39.950000,282.00,2023-11-01,2024-01-04,65.0,4.338462,Full,36000.0
1413,12545.0,44585.0,27150.0,JAWA TIMUR,PACITAN,324.0,160000.0,88.816500,438.60,2023-11-01,2024-01-10,71.0,6.177465,Full,18477760.0
1414,12545.0,44585.0,27150.0,JAWA TIMUR,PACITAN,324.0,160000.0,88.816500,438.60,2023-11-01,2024-01-10,71.0,6.177465,Full,894000.0
1656,12545.0,44587.0,27153.0,JAWA TIMUR,PACITAN,370.0,225000.0,75.759556,460.70,2023-11-01,2024-01-10,71.0,6.488732,Full,17006800.0
1657,12545.0,44587.0,27153.0,JAWA TIMUR,PACITAN,370.0,225000.0,75.759556,460.70,2023-11-01,2024-01-10,71.0,6.488732,Full,1147500.0
3293,12530.0,44517.0,27149.0,JAWA TIMUR,PACITAN,309.0,140000.0,99.045536,448.75,2023-11-01,2024-01-11,72.0,6.232639,Full,1080000.0
3294,12530.0,44517.0,27149.0,JAWA TIMUR,PACITAN,309.0,140000.0,99.045536,448.75,2023-11-01,2024-01-11,72.0,6.232639,Full,19399250.0
4583,12537.0,44544.0,28922.0,SULAWESI BARAT,MAMUJU UTARA,330.0,362738.0,59.133590,650.00,2024-01-18,2024-02-14,28.0,23.214286,Failed,0.0


In [326]:
# Feature engineering
df['days'] = (pd.to_datetime(df['harvested_at']) - pd.to_datetime(df['started_at'])).dt.days + 1
df['biomass'] = df['weight'] * df['size']
df['revenue'] = df['biomass'] * df['selling_price']
df['average_body_weight'] = df['weight'] / df['size']

In [363]:
# Convert categorical columns to numeric
label_encoder = LabelEncoder()
df['province'] = label_encoder.fit_transform(df['province'])
df['regency'] = label_encoder.fit_transform(df['regency'])
df['start'] = df['started_at'].astype(str)

In [364]:
# Select features and target variables
features = ['days', 'total_seed', 'pond_id', 'farm_id', 'cycle_id', 'province', 'regency']
target_sr = 'SR'
target_abw = 'average_body_weight'
target_biomass = 'biomass'
target_revenue = 'revenue'

In [365]:
# Handle missing values
df = df.dropna(subset=[target_sr, target_abw, target_biomass, target_revenue])


In [366]:
# Split the data into training and testing sets
X = df[features]
y_sr = df[target_sr]
y_abw = df[target_abw]
y_biomass = df[target_biomass]
y_revenue = df[target_revenue]

test_size = 0.2
if len(df) < 5:
    test_size = 0.5

X_train, X_test, y_sr_train, y_sr_test = train_test_split(X, y_sr, test_size=test_size, random_state=42)
_, _, y_abw_train, y_abw_test = train_test_split(X, y_abw, test_size=test_size, random_state=42)
_, _, y_biomass_train, y_biomass_test = train_test_split(X, y_biomass, test_size=test_size, random_state=42)
_, _, y_revenue_train, y_revenue_test = train_test_split(X, y_revenue, test_size=test_size, random_state=42)


In [367]:
# Initialize models
model_sr = RandomForestRegressor(random_state=42)
model_abw = RandomForestRegressor(random_state=42)
model_biomass = RandomForestRegressor(random_state=42)
model_revenue = RandomForestRegressor(random_state=42)

In [368]:
# Train models
model_sr.fit(X_train, y_sr_train)
model_abw.fit(X_train, y_abw_train)
model_biomass.fit(X_train, y_biomass_train)
model_revenue.fit(X_train, y_revenue_train)

RandomForestRegressor(random_state=42)

In [369]:
# Make predictions
y_sr_pred = model_sr.predict(X_test)
y_abw_pred = model_abw.predict(X_test)
y_biomass_pred = model_biomass.predict(X_test)
y_revenue_pred = model_revenue.predict(X_test)

In [370]:
# Evaluate models
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    return mae, mse, r2

mae_sr, mse_sr, r2_sr = evaluate_model(y_sr_test, y_sr_pred)
mae_abw, mse_abw, r2_abw = evaluate_model(y_abw_test, y_abw_pred)
mae_biomass, mse_biomass, r2_biomass = evaluate_model(y_biomass_test, y_biomass_pred)
mae_revenue, mse_revenue, r2_revenue = evaluate_model(y_revenue_test, y_revenue_pred)


In [371]:
# Print evaluation results
print(f"Survival Rate - MAE: {mae_sr}, MSE: {mse_sr}, R2: {r2_sr}")
print(f"Average Body Weight - MAE: {mae_abw}, MSE: {mse_abw}, R2: {r2_abw}")
print(f"Biomass - MAE: {mae_biomass}, MSE: {mse_biomass}, R2: {r2_biomass}")
print(f"Revenue - MAE: {mae_revenue}, MSE: {mse_revenue}, R2: {r2_revenue}")


Survival Rate - MAE: 5.431224397189817, MSE: 119.9291972556394, R2: 0.8119549352993953
Average Body Weight - MAE: 2.6438543998427417, MSE: 205.93592878550996, R2: 0.383010809855993
Biomass - MAE: 19913.884166960575, MSE: 1653635433.6924684, R2: 0.9999542276260586
Revenue - MAE: 23342936436157.273, MSE: 4.681598028033489e+27, R2: 0.11168582934187199


In [372]:
# Save predictions to CSV
predictions = pd.DataFrame({
    'cycle_id': X_test['cycle_id'],
    'predicted_sr': y_sr_pred.round(2),
    'predicted_abw': y_abw_pred.round(2),
    'predicted_biomass': y_biomass_pred.round(2),
    'predicted_revenue': y_revenue_pred.round(2)
})
predictions.to_csv('output/prediction.csv', index=False)

## 4. Infer what features/variables that important to make predictions

In [373]:
# Get feature importances from each model
feature_importances_sr = model_sr.feature_importances_
feature_importances_abw = model_abw.feature_importances_
feature_importances_biomass = model_biomass.feature_importances_
feature_importances_revenue = model_revenue.feature_importances_

# Create a DataFrame to display feature importances
features = ['days', 'total_seed', 'pond_id', 'farm_id', 'cycle_id', 'province', 'regency']
importance_df = pd.DataFrame({
    'Feature': features,
    'Importance_SR': feature_importances_sr,
    'Importance_ABW': feature_importances_abw,
    'Importance_Biomass': feature_importances_biomass,
    'Importance_Revenue': feature_importances_revenue
})

# Sort features by importance for each model
importance_df = importance_df.sort_values(by=['Importance_SR', 'Importance_ABW', 'Importance_Biomass', 'Importance_Revenue'], ascending=False)

# Display the feature importances
importance_df

,Feature,Importance_SR,Importance_ABW,Importance_Biomass,Importance_Revenue
1,total_seed,0.318583,0.168109,9.948448e-01,0.419901
2,pond_id,0.255979,0.388858,4.020129e-03,0.191069
4,cycle_id,0.247982,0.296254,6.007977e-04,0.242432
0,days,0.175927,0.145545,5.331913e-04,0.146158
3,farm_id,0.001142,0.000004,3.445045e-08,0.000015
6,regency,0.000210,0.001227,5.192957e-07,0.000202
5,province,0.000178,0.000004,4.997011e-07,0.000224
